In [ ]:
import pandas as pd


def inputdata(path):
    data = pd.read_csv(path, header=0, sep=",", encoding="utf-8")
    return data


def outputdata(path, data, is_index=False):
    data.to_csv(path, index=is_index, header=True, sep=",", mode="w", encoding="utf-8")


def transcolname(df, column_mapping):
    df.rename(columns=column_mapping, inplace=True)
    return df



def trans_datetime(df):
    ret_df = pd.DataFrame()
    dt = df["Date"]
    ret_df["year"] = dt.transform(lambda x: int(x.split("-")[0]))
    ret_df["month"] = dt.transform(lambda x: int(x.split("-")[1]))
    ret_df["day"] = dt.transform(lambda x: int(x.split("-")[2][:2]))
    df = pd.concat([df, ret_df], axis=1)
    unique_dates = pd.Series(df["Date"].unique()).sort_values().reset_index(drop=True)
    date_mapping = {date: rank + 1 for rank, date in enumerate(unique_dates)}
    df["Date"] = df["Date"].map(date_mapping)
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    # minTime = df["Date"].min()
    # df["Date"] = ((df["Date"] - minTime) / pd.Timedelta(days=1)).astype(int)
    return df


def processing_feature():
    # 读取数据
    data = inputdata("train.csv")
    column_mapping = {
        "股票代码": "StockCode",
        "日期": "Date",
        "开盘": "Open",
        "收盘": "Close",
        "最高": "High",
        "最低": "Low",
        "成交量": "Volume",
        "成交额": "Turnover",
        "振幅": "Amplitude",
        "涨跌额": "PriceChange",
        "换手率": "TurnoverRate",
        "涨跌幅": "PriceChangePercentage",
    }
    data = transcolname(data, column_mapping)
    data.drop(columns=["PriceChangePercentage"], inplace=True)
    data = trans_datetime(data)

    return data


feature = processing_feature()

outputdata("feature.csv", feature)


In [ ]:
import pandas as pd
import pickle
import torch


def inputdata(path):
    data = pd.read_csv(path, header=0, sep=",", encoding="utf-8")
    return data


feature = inputdata("feature.csv")


def process_data(npdf, stp=32):
    ret = []
    for i in range(npdf.shape[0] - stp):
        train_seq = npdf[i : i + stp]
        train_label = npdf[i + stp]
        train_seq = torch.FloatTensor(train_seq)
        train_label = torch.FloatTensor(train_label).view(-1)
        ret.append((train_seq, train_label))

    return ret


column_names = feature.columns.tolist()
stockcodes = feature["StockCode"].drop_duplicates().tolist()

train_data = []
for stockcode in stockcodes:
    stock_data = feature[feature["StockCode"] == stockcode]
    max_date = stock_data["Date"].max()
    min_date = stock_data["Date"].min()
    stock_data = stock_data.values
    if len(stock_data) < 32:
        continue
    train_data += process_data(stock_data, stp=32)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 模型参数
input_size = 1
hidden_size = 500
num_layers = 4
output_size = 1
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def __del__(self):
        del self.hidden_cell

    def forward(self, x):
        out, self.hidden_cell = self.lstm(x, self.hidden_cell)
        out = self.fc(out[:, -1, :])
        return out


def train_model(train_data, i, num_epochs=50):
    if len(train_data) == 0:
        return LSTMModel(1, hidden_size, num_layers, output_size, dropout).to(device)

    train_data = [(x.to(device), y.to(device)) for x, y in train_data]

    # 调整每个输入序列的维度为3D
    X_train_tensor = torch.stack([x for x, _ in train_data])
    y_train_tensor = torch.stack([y[i] for _, y in train_data])

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    model = LSTMModel(
        len(train_data[0][0][0]), hidden_size, num_layers, output_size, dropout
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

    for epoch in range(num_epochs):
        # print(epoch)
        tot_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            model.hidden_cell = (
                torch.zeros(model.num_layers, batch_X.size(0), model.hidden_size).to(
                    device
                ),
                torch.zeros(model.num_layers, batch_X.size(0), model.hidden_size).to(
                    device
                ),
            )
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            tot_loss += loss.item()
            loss.backward()
            optimizer.step()
        scheduler.step()
        print("Epoch [{}/{}], Loss: {:.4f}".format(epoch + 1, num_epochs, tot_loss))
    return model


colname2index = {x: i for i, x in enumerate(column_names)}
model_i = train_model(train_data, colname2index["Close"] + 2, num_epochs=5)
model_name = "model_Close0.bin"
pickle.dump(model_i, open(model_name, "wb"))


In [ ]:
import pandas as pd
import numpy as np
import pickle
import torch

pred_len = 32


def inputdata(path):
    data = pd.read_csv(path, header=0, sep=",", encoding="utf-8")
    return data


def outputdata(path, data, is_index=False):
    data.to_csv(path, index=is_index, header=True, sep=",", mode="w", encoding="utf-8")


def transcolname(df, column_mapping):
    df.rename(columns=column_mapping, inplace=True)
    return df


def trans_datetime(df):
    ret_df = pd.DataFrame()
    dt = df["Date"]
    ret_df["year"] = dt.transform(lambda x: int(x.split("-")[0]))
    ret_df["month"] = dt.transform(lambda x: int(x.split("-")[1]))
    ret_df["day"] = dt.transform(lambda x: int(x.split("-")[2][:2]))
    df = pd.concat([df, ret_df], axis=1)
    unique_dates = pd.Series(df["Date"].unique()).sort_values().reset_index(drop=True)
    date_mapping = {date: rank + 1 for rank, date in enumerate(unique_dates)}
    df["Date"] = df["Date"].map(date_mapping)
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    # minTime = df["Date"].min()
    # df["Date"] = ((df["Date"] - minTime) / pd.Timedelta(days=1)).astype(int)
    return df


def processing_feature_test():
    # 读取数据
    # data1 = inputdata("./data/train.csv")
    data = inputdata("test.csv")
    column_mapping = {
        "股票代码": "StockCode",
        "日期": "Date",
        "开盘": "Open",
        "收盘": "Close",
        "最高": "High",
        "最低": "Low",
        "成交量": "Volume",
        "成交额": "Turnover",
        "振幅": "Amplitude",
        "涨跌额": "PriceChange",
        "换手率": "TurnoverRate",
        "涨跌幅": "PriceChangePercentage",
    }
    # data = pd.concat([data1, data2], axis=0)
    data = transcolname(data, column_mapping)
    data.drop(columns=["PriceChangePercentage"], inplace=True)
    # stockcodes = data["StockCode"].drop_duplicates().tolist()
    data = trans_datetime(data)
    max_date = data["Date"].max()
    data = data[data["Date"] > max_date - pred_len]

    return data


data = processing_feature_test()
max_date = data["Date"].max()


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 模型参数
input_size = 1
hidden_size = 500
num_layers = 4
output_size = 1
dropout = 0.2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def __del__(self):
        del self.hidden_cell

    def forward(self, x):
        out, self.hidden_cell = self.lstm(x, self.hidden_cell)
        out = self.fc(out[:, -1, :])
        return out


PREDICT_COLS = [
    "Open",
    "Close",
    "High",
    "Low",
    "Volume",
    "Turnover",
    "Amplitude",
    "PriceChange",
    "TurnoverRate",
]
column_names = data.columns.tolist()
colname2index = {x: i for i, x in enumerate(column_names)}
stockcodes = data["StockCode"].drop_duplicates().tolist()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_preds = []


def process_data_predict(data, stockcode):
    data = data[data["StockCode"] == stockcode]
    data = np.array([data.values])
    return torch.tensor(data, dtype=torch.float32).to(device)


for stockcode in stockcodes:

    temp_preds = {}

    # 预测当前变量
    predict_data = process_data_predict(data, stockcode).to(device)
    model_name = "model_Close0.bin"
    estimator = pickle.load(open(model_name, "rb"))
    estimator.hidden_cell = (
        torch.zeros(
            estimator.num_layers, predict_data.size(0), estimator.hidden_size
        ).to(device),
        torch.zeros(
            estimator.num_layers, predict_data.size(0), estimator.hidden_size
        ).to(device),
    )
    with torch.no_grad():
        predi = estimator(predict_data)
    pred = predi[-1].cpu().detach().numpy()

    all_preds.append((stockcode, pred))


pricechangerate = []
for i in range(len(all_preds)):
    stockcode, pred = all_preds[i]
    preClose = data[(data["StockCode"] == stockcode) & (data["Date"] == max_date)][
        "Close"
    ].values[0]
    pricechangerate.append((stockcode, (pred - preClose) / preClose * 100))
pricechangerate = sorted(pricechangerate, key=lambda x: x[1], reverse=True)
pred_top_10_max_target = [x[0] for x in pricechangerate[:10]]
pred_top_10_min_target = [x[0] for x in pricechangerate[-10:]]
data = {
    "涨幅最大股票代码": pred_top_10_max_target,
    "涨幅最小股票代码": pred_top_10_min_target,
}

df = pd.DataFrame(data)
df.to_csv("result.csv", index=False)
